In [40]:
%load_ext autoreload
%autoreload 2
from src.data.processing import tokenize_dataset
import transformers
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments,DataCollatorForSeq2Seq, AutoTokenizer
import evaluate
import datasets
import numpy as np

In [3]:
data = datasets.load_dataset("Samsung/samsum")

z:\NLP_project\venv\lib\site-packages\datasets\load.py:1491: FutureWarning: The repository for Samsung/samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Samsung/samsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
MODEL_NAME = "google-t5/t5-small"
DEVICE = "cuda"
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, device_map=DEVICE)
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME, device_map=DEVICE)

In [5]:
tokenized_dataset = tokenize_dataset(data, tokenizer, True)

Max source length: 255
Max target length: 45


Map: 100%|██████████| 819/819 [00:00<00:00, 3740.34 examples/s]


In [6]:
tokenized_dataset.set_format(type="torch")

In [6]:
rouge = datasets.load_metric("rouge")
rouge


C:\Users\Lucas\AppData\Local\Temp\ipykernel_8920\2953629268.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric("rouge")
z:\NLP_project\venv\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [25]:
training_args = Seq2SeqTrainingArguments(
    output_dir=".output/t5-summarizer",
    learning_rate=1e-3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
	remove_unused_columns=False 
)

data_collator = DataCollatorForSeq2Seq(tokenizer,model,
	return_tensors="pt",
	label_pad_token_id=-100,
	pad_to_multiple_of=8)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator
)

# Evaluate the model
eval_results = trainer.predict(tokenized_dataset["test"].select(range(10)))

z:\NLP_project\venv\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00, 39.99it/s]


In [43]:
out = model.generate(tokenized_dataset["test"].select(range(10))["input_ids"].to("cuda"),max_new_tokens=100)

In [44]:
print(data["test"][1]["dialogue"])

Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)


In [42]:
tokenizer.decode(tokenized_dataset["test"]["input_ids"][0],skip_special_tokens=True)

"summarize: Hannah: Hey, do you have Betty's number? Amanda: Lemme check Hannah: file_gif> Amanda: Sorry, can't find it. Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well Hannah: file_gif> Amanda: Don't be shy, he's very nice Hannah: If you say so.. Hannah: I'd rather you texted him Amanda: Just text him  Hannah: Urgh.. Alright Hannah: Bye Amanda: Bye bye"